# 1. Importing Libraries and Loading Data

In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as ps
import mysql.connector as ms

import warnings
warnings.filterwarnings('ignore')

In [123]:
df_info_clients = pd.read_csv('data/clien_info_right.csv')
df_info_clients.drop(['Unnamed: 0'], axis=1, inplace=True)
df_info_clients.head()

,amostra,id_cliente,id_unico_cliente,item_id,cep_cliente,cidade_cliente,estado_cliente,id_pedido,status_pedido,horario_pedido,...,pagamento_sequencial,pagamento_tipo,pagamento_prestacoes,pagamento_valor,review,categoria,categoria_ingles,cep_vendedor,cidade_vendedor,estado_vendedor
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,1,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2020-01-30,...,1,credit_card,2,1092.36,4,moveis_escritorio,office_furniture,8577,itaquaquecetuba,SP
1,1,8912fc0c3bbf1e2fbf35819e21706718,9eae34bbd3a474ec5d07949ca7de67c0,1,68030,santarem,PA,c1d2b34febe9cd269e378117d6681172,delivered,2022-03-26,...,1,credit_card,1,211.33,1,moveis_escritorio,office_furniture,8577,itaquaquecetuba,SP
2,2,8912fc0c3bbf1e2fbf35819e21706718,9eae34bbd3a474ec5d07949ca7de67c0,2,68030,santarem,PA,c1d2b34febe9cd269e378117d6681172,delivered,2021-06-06,...,1,credit_card,1,992.38,1,moveis_escritorio,office_furniture,8577,itaquaquecetuba,SP
3,3,f0ac8e5a239118859b1734e1087cbb1f,3c799d181c34d51f6d44bbbc563024db,1,92480,nova santa rita,RS,b1a5d5365d330d10485e0203d54ab9e8,delivered,2020-10-07,...,1,credit_card,7,709.16,3,moveis_escritorio,office_furniture,8577,itaquaquecetuba,SP
4,4,6bc8d08963a135220ed6c6d098831f84,23397e992b09769faf5e66f9e171a241,1,25931,mage,RJ,2e604b3614664aa66867856dba7e61b7,delivered,2020-04-19,...,1,credit_card,10,284.44,4,moveis_escritorio,office_furniture,8577,itaquaquecetuba,SP


## 1.1) Translating the dataframe

- **Amostra:**  id

- **id_cliente:** client_id

- **id_unico_cliente:**: client_specific_id

- **item-id:** item_id

- **cep_cliente:** client _postal_code

- **cidade cliente:** client_city

- **estado_cliente:** client_state

- **id_pedido:** order_id

- **status_pedido:** order_status

- **horario pedido:** order_date

- **pedido_aprovado:** order_approved (date and time that the order was approved)

- **pedido_entregador:** order_dliveryman (date and time that the order was leaving for delivery)

- **pedido_entregue:** order_delivery (date and time that the order was delivery to the client)

- **data_estimada:** order_estimated (estimate date that the order will arrive to the client)

- **id_vendedor:** salesman_id

- **preco:** order_price (price of the order in reais (R$))

- **frete:** delivery_price

- **pagamento_sequencial:** sequence_pay

- **pagamento_tipo:** paymeny_type

- **pagamento_prestacoes:**  divided_pay (number of parcels that the client will pay the order's value)

- **pagamento_valor:** ntotal_price (order's price + dlivery's price)

- **review:** review (stars rating of the client)

- **categoria:** category (product's category)

- **categoria_ingles:** category_english (product's category in english)

- **cep_vendedor:** salesman_postal

- **cidade_vendedor:** salesman_city

- **estado_vendedor:** salesman_state

In [124]:
#Rename the columns
new_columns = [ 'id',  'client_id', 'client_specific_id', 'item_id', 'client _postal_code', 'client_city', 'client_state', 'order_id', 'order_status', 'order_date', 'order_approved', 'order_dliveryman',
               'order_delivery', 'order_estimated', 'salesman_id', 'order_price', 'delivery_price', 'sequence_pay', 'paymeny_type', 'divided_pay', 'total_price', 'review', 'category', 'category_english',
               'salesman_postal', 'salesman_city', 'salesman_state']

df_info_clients.columns = new_columns

# 2) Data Caracteristics

In [125]:
print('Number of Rows: {}'.format( df_info_clients.shape[0] ) )
print( 'Number of Columns: {}'.format( df_info_clients.shape[1] ) )

Number of Rows: 116581
Number of Columns: 27


In [126]:
#type of the columns
print('Data Types')
df_info_clients.dtypes

Data Types


id                       int64
client_id               object
client_specific_id      object
item_id                  int64
client _postal_code      int64
client_city             object
client_state            object
order_id                object
order_status            object
order_date              object
order_approved          object
order_dliveryman        object
order_delivery          object
order_estimated         object
salesman_id             object
order_price            float64
delivery_price         float64
sequence_pay             int64
paymeny_type            object
divided_pay              int64
total_price            float64
review                   int64
category                object
category_english        object
salesman_postal          int64
salesman_city           object
salesman_state          object
dtype: object

In [127]:
# Convert datas from object to date_time
df_info_clients[["order_date"]] = df_info_clients[["order_date"]].apply(pd.to_datetime)
df_info_clients[["order_approved"]] = df_info_clients[["order_approved"]].apply(pd.to_datetime)
df_info_clients[["order_dliveryman"]] = df_info_clients[["order_dliveryman"]].apply(pd.to_datetime)
df_info_clients[["order_delivery"]] = df_info_clients[["order_delivery"]].apply(pd.to_datetime)
df_info_clients[["order_estimated"]] = df_info_clients[["order_estimated"]].apply(pd.to_datetime)

In [128]:
print("Check Null Values")
df_info_clients.isna().sum()

Check Null Values


id                     0
client_id              0
client_specific_id     0
item_id                0
client _postal_code    0
client_city            0
client_state           0
order_id               0
order_status           0
order_date             0
order_approved         0
order_dliveryman       0
order_delivery         0
order_estimated        0
salesman_id            0
order_price            0
delivery_price         0
sequence_pay           0
paymeny_type           0
divided_pay            0
total_price            0
review                 0
category               0
category_english       0
salesman_postal        0
salesman_city          0
salesman_state         0
dtype: int64

In order to analyse the status of the null values

In [129]:
mask = df_info_clients['order_approved'].isnull() | df_info_clients['order_dliveryman'].isnull() | df_info_clients['order_delivery'].isnull()

# group the rows by order_status and count the ids for each order_status
df_info_clients[mask].groupby('order_status')['id'].count()

Series([], Name: id, dtype: int64)

It is possible to see that only 23 of the null values are erros, because the status is "delivered" but they do not have: 
- the date of order approval 

or 
- the date that the order was leaving for delivery 

or 
- the date that the order was delivery to the client

For this reason, I will delete these values.

In [130]:
df_info_clients[(df_info_clients['order_approved'].isnull() | df_info_clients['order_dliveryman'].isnull() | df_info_clients['order_delivery'].isnull()) & (df_info_clients['order_status'] == 'delivered')].iloc[:, 8:14]

,order_status,order_date,order_approved,order_dliveryman,order_delivery,order_estimated


In [131]:
# deleting null values of 'delivered' status

df_info_clients = df_info_clients[(df_info_clients['order_approved'].notnull() & df_info_clients['order_dliveryman'].notnull() & df_info_clients['order_delivery'].notnull()) | (df_info_clients['order_status'] != 'delivered')]

# 3) Classifying  customers

## 3.1) ABC method

Classifying customers may assist you to streamline your sales activities and help you focus on achieving the desired business goals. The ABC method is the most commonly used.

A - represents the ‘top grade’ or ‘ideal’ client in your business. They are 20% of the clients and generate more than 80% of the biling.

B - is a brilliant business customer. Represent 15% of the billing and they are 30% of the clients

C - low revelance for the business. The majority of the cliets correspond to only 5% of the revenue

In [132]:
print("There are "+ str(len(df_info_clients)) + " sales and " +
    str(len(df_info_clients['client_id'].unique())) + 
    " different clients " )

There are 116581 sales and 97255 different clients 


In [133]:
#Clients that bought more than once
counts = df_info_clients['client_id'].value_counts()
id_good_clients = counts[counts>1].index.tolist()

print( str(len(id_good_clients)) +
    " clients bought more than once, which represents " +
    str(round(((len(id_good_clients)/len(df_info_clients['client_id'].unique()))*100),2)) +
    "%")


12729 clients bought more than once, which represents 13.09%


In [134]:
#separate the dataframe for only the good clients
df_good_clients = df_info_clients[df_info_clients['client_id'].isin(id_good_clients)]

In [135]:
print("The clients that bought more than once represents " +
str(round((df_good_clients['order_price'].sum()/df_info_clients['order_price'].sum())*100,2))+
"% of the billing")

The clients that bought more than once represents 27.48% of the billing


*DESCONSIDERED*

## 3.2) Creating Features

I will create a dataframe of clients with the following features:
- Client ID
- Client State
- Client city 
- Number of orders
- Frquency of orders per month: number of orders divided by the difference between the date of the first order and the last one
- Mean of the price of the order
- Mean of the reviews

In [136]:
df_info_clients['order_date2'] = df_info_clients['order_date'] 
df_clients = df_info_clients.groupby(['client_id', 'client_state', 'client_city']).agg(
    {'client_id': 'count', 'order_price': 'mean', 'review': 'mean', 'order_date': 'min', 'order_date2': 'max'}).rename(
    columns={'client_id': 'number_orders', 'order_price': 'mean_price', 'review': 'mean_review', 'order_date': 'min_order_date', 'order_date2': 'max_order_date'}).reset_index()
df_clients.head()

,client_id,client_state,client_city,number_orders,mean_price,mean_review,min_order_date,max_order_date
0,00012a2ce6f8dcda20d059ce98491703,SP,osasco,1,1070.75,1.0,2021-06-19,2021-06-19
1,000161a058600d5901f007fab4c27140,MG,itapecerica,1,50.74,4.0,2022-12-25,2022-12-25
2,0001fd6190edaaf884bcaf3d49edf079,ES,nova venecia,1,457.14,5.0,2019-09-19,2019-09-19
3,0002414f95344307404f0ace7a26f1d5,MG,mendonca,1,138.65,5.0,2020-03-28,2020-03-28
4,000379cdec625522490c315e70c7a9fb,SP,sao paulo,1,695.97,4.0,2022-12-31,2022-12-31


In [137]:
df_clients['max_order_date'] = pd.to_datetime(df_clients['max_order_date'])
df_clients['min_order_date'] = pd.to_datetime(df_clients['min_order_date'])

In [138]:
df_clients['diff_days'] = (df_clients['max_order_date'] - df_clients['min_order_date']).dt.days

In [139]:
df_clients['interval'] = None

In [140]:
#defyne the number of days between purchase
df_clients['interval'] = round(df_clients['diff_days'] / df_clients['number_orders'],2)

In [141]:
df_clients[df_clients['number_orders']>1]

,client_id,client_state,client_city,number_orders,mean_price,mean_review,min_order_date,max_order_date,diff_days,interval
12,00066ccbe787a588c52bd5ff404590e3,RS,novo hamburgo,4,681.000000,1.0,2019-08-30,2022-02-21,906,226.50
15,000bf8121c3412d3057d32371c5d3395,SP,jacarei,2,913.140000,5.0,2020-01-20,2020-02-27,38,19.00
16,000e943451fc2788ca6ac98a682f2f49,RS,colorado,4,563.247500,4.0,2019-09-21,2022-07-20,1033,258.25
22,001051abfcfdbed9f87b4266213a5df1,SP,sao paulo,3,622.826667,4.0,2019-09-19,2022-06-17,1002,334.00
28,001450ebb4a77efb3d68be5f7887cb1e,SP,ibiuna,2,560.395000,2.0,2019-04-22,2021-02-25,675,337.50
...,...,...,...,...,...,...,...,...,...,...
97230,fff168ca1f8a1d2e8e2108b231a68a8c,SP,sao paulo,3,310.363333,5.0,2019-04-01,2022-09-08,1256,418.67
97236,fff3a5c6d542d52b05f7e4518adf996e,MS,anhandui,3,715.480000,5.0,2019-09-11,2022-11-11,1157,385.67
97247,fff93c1da78dafaaa304ff032abc6205,SP,araraquara,3,321.016667,5.0,2019-07-08,2021-04-12,644,214.67
97251,fffecc9f79fd8c764f843e9951b11341,RS,parobe,3,629.076667,3.0,2019-12-09,2022-07-28,962,320.67


In [142]:
df_info_clients['client_id'].value_counts()

270c23a11d024a44c896d1894b261a83    63
13aa59158da63ba0e93ec6ac2c07aacb    38
9af2372a1e49340278e7c1ef8d749f34    29
92cd3ec6e2d643d4ebd0e3d6238f69e2    26
d22f25a9fadfb1abbc2e29395b1239f4    24
                                    ..
f072b86e7d9e54f95e3dc4854fddeb3d     1
7facd235e6759b64f900b3a24f71a2a6     1
20fd1d3f2cd5764f96b64c33263a57fd     1
33a3023585bff9c06044bdcd92c95b4b     1
c61336d6c835632048f973d2f2db598d     1
Name: client_id, Length: 97255, dtype: int64